In [1]:
import pandas as pd 
import git
import numpy as np
import geopandas as gpd
import plotly as px
import networkx as nx
from datetime import datetime as dt
from vehicle_stream_pipeline.utils import get_shortest_ride, get_hotspots, calculate_drives, calculate_graph, add_drone_flights, poi_poly, generateRideSpecs, find_id_for_name

repo = git.Repo(".", search_parent_directories=True).git.rev_parse(
    "--show-toplevel")

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv(r'/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/cleaning/data_cleaned.csv')
# df.to_excel(r'/Users/ericchittka/Downloads/data_cleaned.xlsx')

/var/folders/kw/t27bnrpn52lfl7x3g64tdw1r0000gn/T/ipykernel_84168/1512512131.py:1: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/cleaning/data_cleaned.csv')


In [4]:
df_c = df[df['state']=='completed']

In [5]:
df_c[['created_at', 'scheduled_to', 'dispatched_at', 'arriving_push', 'vehicle_arrived_at', 'earliest_pickup_expectation', 'pickup_first_eta', 'pickup_eta', 'pickup_at', 'dropoff_first_eta', 'dropoff_eta', 'dropoff_at', 'updated_at']] = df_c[['created_at', 'scheduled_to', 'dispatched_at', 'arriving_push', 'vehicle_arrived_at', 'earliest_pickup_expectation', 'pickup_first_eta', 'pickup_eta', 'pickup_at', 'dropoff_first_eta', 'dropoff_eta', 'dropoff_at', 'updated_at']].apply(pd.to_datetime)


/var/folders/kw/t27bnrpn52lfl7x3g64tdw1r0000gn/T/ipykernel_84168/1296458390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c[['created_at', 'scheduled_to', 'dispatched_at', 'arriving_push', 'vehicle_arrived_at', 'earliest_pickup_expectation', 'pickup_first_eta', 'pickup_eta', 'pickup_at', 'dropoff_first_eta', 'dropoff_eta', 'dropoff_at', 'updated_at']] = df_c[['created_at', 'scheduled_to', 'dispatched_at', 'arriving_push', 'vehicle_arrived_at', 'earliest_pickup_expectation', 'pickup_first_eta', 'pickup_eta', 'pickup_at', 'dropoff_first_eta', 'dropoff_eta', 'dropoff_at', 'updated_at']].apply(pd.to_datetime)


In [7]:
# filter wrong ordering: created_at
df_test = df_c.loc[(df_c.created_at > df_c.scheduled_to) | (df_c.created_at > df_c.dispatched_at)
| (df_c.created_at > df_c.arriving_push) | (df_c.created_at > df_c.vehicle_arrived_at)
| (df_c.created_at > df_c.earliest_pickup_expectation) | (df_c.created_at > df_c.pickup_first_eta)
| (df_c.created_at > df_c.pickup_eta) | (df_c.created_at > df_c.pickup_at)
| (df_c.created_at > df_c.dropoff_first_eta) | (df_c.created_at > df_c.dropoff_eta)
| (df_c.created_at > df_c.dropoff_at) | (df_c.created_at > df_c.updated_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [8]:
# filter wrong ordering: scheduled_to
df_test = df_c.loc[(df_c.scheduled_to < df_c.dispatched_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [9]:
# filter all timestamps that are not on the same date than scheduled_to 
# automatically validated if all other timestamps are on the same day  
df_test = df_c.loc[(df_c.dispatched_at.dt.day != df_c.scheduled_to.dt.day) | (df_c.arriving_push.dt.day != df_c.scheduled_to.dt.day) 
| (df_c.vehicle_arrived_at.dt.day != df_c.scheduled_to.dt.day) | (df_c.earliest_pickup_expectation.dt.day != df_c.scheduled_to.dt.day) 
| (df_c.pickup_first_eta.dt.day != df_c.scheduled_to.dt.day) | (df_c.pickup_eta.dt.day != df_c.scheduled_to.dt.day) | (df_c.pickup_at.dt.day != df_c.scheduled_to.dt.day) 
| (df_c.dropoff_first_eta.dt.day != df_c.scheduled_to.dt.day) | (df_c.dropoff_eta.dt.day != df_c.scheduled_to.dt.day) | (df_c.dropoff_at.dt.day != df_c.scheduled_to.dt.day)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index
971,657e96a6-86bc-4246-b183-13475ab7f754,41d909e4-0978-4687-b01a-8596335a1a94,6171.0,2.0,9.62,9.62,9.62,False,STANDARD,12005,4014,completed,8946.0,2022-01-07 23:36:16,2022-01-07 23:36:16,2022-01-07 23:36:16,566.0,2022-01-07 23:44:22,2022-01-07 23:45:42,2022-01-07 23:39:16,2022-01-07 23:48:33,2022-01-07 23:47:03,2022-01-07 23:45:44,2022-01-08 00:02:10,2022-01-08 00:00:41,2022-01-07 23:58:49,2022-01-07 23:58:55,-100.0,386.0,2.0,785.0,1171.0,741.0,430.0,1.06,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2114,53a70562-72f3-4382-bcb4-a704f1af5b92,0e82a6ae-7d07-45b2-9c0c-10a03dfa37ff,5101.0,1.0,4.86,4.33,4.33,False,BAHN_CARD,1005,4013,completed,NaN,2022-01-28 22:50:23,2022-01-28 23:50:00,2022-01-28 23:42:00,643.0,2022-01-28 23:49:35,2022-01-28 23:52:43,2022-01-28 23:45:00,2022-01-28 23:55:34,2022-01-28 23:54:04,2022-01-28 23:52:45,2022-01-29 00:04:58,2022-01-29 00:13:46,2022-01-29 00:07:47,2022-01-29 00:07:49,8.0,463.0,2.0,902.0,1365.0,612.0,753.0,1.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2115,4063b52e-0697-47ae-9539-801f78ff308d,b204438a-ca02-4a7b-8251-c4ad74f917d4,6152.0,1.0,5.41,5.41,5.41,False,STANDARD,1005,5006,completed,NaN,2022-01-28 22:48:41,2022-01-28 23:50:00,2022-01-28 23:42:00,655.0,2022-01-28 23:50:41,2022-01-28 23:52:55,2022-01-28 23:45:00,2022-01-28 23:56:31,2022-01-28 23:54:16,2022-01-28 23:52:59,2022-01-29 00:07:51,2022-01-29 00:05:34,2022-01-29 00:01:26,2022-01-29 00:01:27,-46.0,475.0,4.0,507.0,982.0,738.0,244.0,0.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2118,ba18b26e-38b3-425b-9081-ac113e2c49d4,44f61d06-8e79-42c6-9abd-0e85fcaf9d6d,3572.0,3.0,9.79,7.11,7.11,False,VRN,4030,3001,completed,9741.0,2022-01-28 23:56:23,2022-01-28 23:56:23,2022-01-28 23:56:23,246.0,2022-01-28 23:57:03,2022-01-29 00:00:29,2022-01-28 23:59:23,2022-01-29 00:01:05,2022-01-29 00:04:13,2022-01-29 00:04:57,2022-01-29 00:08:06,2022-01-29 00:11:13,2022-01-29 00:10:43,2022-01-29 00:10:45,26.0,66.0,268.0,346.0,412.0,429.0,-17.0,0.81,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,87131775-38f2-4eb1-8986-3c2e4296f61c,b204438a-ca02-4a7b-8251-c4ad74f917d4,6150.0,2.0,8.10,8.10,8.10,False,STANDARD,1005,5006,completed,NaN,2021-11-12 23:11:07,2021-11-12 23:50:00,2021-11-12 23:42:00,1066.0,2021-11-12 23:56:45,2021-11-12 23:59:46,2021-11-12 23:45:00,2021-11-12 23:57:18,2021-11-13 00:00:49,2021-11-13 00:00:02,2021-11-13 00:06:53,2021-11-13 00:10:28,2021-11-13 00:28:40,2021-11-15 11:24:38,1.0,886.0,16.0,1718.0,2604.0,738.0,1866.0,2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18692,f5df0493-0110-4dde-8a73-ac7f87b519f7,41c95331-14bc-4433-a0ce-46017b56aec2,5947.0,4.0,17.23,17.23,17.23,False,STANDARD,4038,16027,completed,NaN,2022-04-29 18:42:33,2022-04-29 23:50:00,2022-04-29 23:42:00,1017.0,2022-04-29 23:54:40,2022-04-29 23:58:57,2022-04-29 23:45:00,2022-04-29 23:56:12,2022-04-30 00:00:05,2022-04-29 23:59:41,2022-04-30 00:04:51,2022-04-30 00:08:26,2022-04-30 00:11:09,2022-04-30 00:12:34

In [10]:
# filter cases where the timestamps are not on the same day because they were at midnight  
# use dropoff_at for comparison (should include all other timestamps)
df_test = df_test.loc[(df_test.dropoff_at - df_test.scheduled_to > pd.Timedelta(minutes=60))]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [11]:
# filter wrong ordering: dispatched_at
df_test = df_c.loc[ (df_c.dispatched_at > df_c.vehicle_arrived_at) | (df_c.dispatched_at > df_c.earliest_pickup_expectation) | (df_c.dispatched_at > df_c.pickup_first_eta)
| (df_c.dispatched_at > df_c.pickup_eta) | (df_c.dispatched_at > df_c.pickup_at) | (df_c.dispatched_at > df_c.dropoff_first_eta) | (df_c.dispatched_at > df_c.dropoff_eta)
| (df_c.dispatched_at > df_c.dropoff_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [12]:
# filter wrong ordering: arriving_push
df_test = df_c.loc[(df_c.arriving_push > df_c.vehicle_arrived_at) | (df_c.arriving_push > df_c.pickup_at) | (df_c.arriving_push > df_c.dropoff_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [13]:
# filter wrong ordering: vehicle_arrived_at
df_test = df_c.loc[(df_c.vehicle_arrived_at > df_c.pickup_at) | (df_c.vehicle_arrived_at > df_c.dropoff_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [14]:
# filter wrong ordering: pickup_at
df_test = df_c.loc[(df_c.pickup_at > df_c.dropoff_at)]
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [15]:
# test the calculations
# pickup_arrival_time
df_test = df_c.loc[((df_c.vehicle_arrived_at - df_c.dispatched_at).dt.seconds != df_c.pickup_arrival_time) 
# arrival_deviation
| (((df_c.vehicle_arrived_at - df_c.arriving_push).dt.seconds -180)!= df_c.arrival_deviation)
# waiting_time
| (((df_c.vehicle_arrived_at - df_c.earliest_pickup_expectation).dt.seconds != df_c.waiting_time) & (df_c.vehicle_arrived_at>df_c.earliest_pickup_expectation))
# filter cases where the values where negative
| (((df_c.vehicle_arrived_at - df_c.earliest_pickup_expectation).dt.seconds - 86400 != df_c.waiting_time) & (df_c.vehicle_arrived_at<df_c.earliest_pickup_expectation))
# boarding_time
| ((df_c.pickup_at - df_c.vehicle_arrived_at).dt.seconds != df_c.boarding_time)
# ride_time
| ((df_c.dropoff_at - df_c.pickup_at).dt.seconds != df_c.ride_time)
# trip_time 
| ((df_c.ride_time + df_c.waiting_time) != df_c.trip_time)
# delay
| ((df_c.trip_time - df_c.shortest_ridetime) != df_c.delay)
] 
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index


In [16]:
# filiter the biggest outliers 
# pickup_arrival_time
df_test = df_c.loc[(df_c.pickup_arrival_time >= 10000) 
# arrival_deviation
| (df_c.arrival_deviation >= 2000)
# waiting_time
| (df_c.waiting_time >= 3000) 
# boarding_time
| (df_c.boarding_time >= 2000) 
# ride_time
| (df_c.ride_time >= 5000) 
# trip_time 
| (df_c.trip_time >= 5000) 
# delay
| (df_c.delay >= 5000) 
] 
df_test

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index
2566,defe9bb3-1f26-4044-91ac-f555140e94c8,76f7bb28-ae72-4d6b-83f5-ce252a641f81,5435.0,1.0,4.65,4.65,4.65,False,STANDARD,11003,7001,completed,NaN,2021-11-06 16:38:18,2021-11-07 07:00:00,2021-11-07 06:52:00,234.0,2021-11-07 06:52:54,2021-11-07 06:55:54,2021-11-07 06:55:00,2021-11-07 07:40:36,2021-11-07 07:42:44,2021-11-07 07:31:18,2021-11-07 07:51:08,2021-11-07 07:53:16,2021-11-07 07:31:25,2021-11-15 11:24:38,0.0,54.0,2124.0,7.0,61.0,652.0,-591.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4084,0cbf29b1-2644-40fd-9c6f-03073fae4dea,ce755d22-cfba-49c3-b37a-623f3c52cf7d,2079.0,1.0,2.95,2.55,2.55,False,BAHN_CARD,5003,4026,completed,5585.0,2021-10-07 20:29:11,2021-10-07 20:29:11,2021-10-07 20:29:11,6168.0,2021-10-07 22:08:59,2021-10-07 22:11:59,2021-10-07 20:32:11,2021-10-07 22:11:28,2021-10-07 22:18:10,2021-10-07 22:13:16,2021-10-07 22:17:01,2021-10-07 22:23:29,2021-10-07 22:17:47,2021-11-15 11:24:38,0.0,5988.0,77.0,271.0,6259.0,249.0,6010.0,1.09,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5013,3aff5be2-420c-4e37-bf34-fc8304136e60,729c1dee-ab62-43a2-abf9-5e6320fb2338,796.0,1.0,2.40,2.40,2.40,False,STANDARD,1001,1004,completed,4620.0,2021-09-02 17:29:01,2021-09-02 17:29:01,2021-09-02 17:29:01,5940.0,2021-09-02 19:05:01,2021-09-02 19:08:01,2021-09-02 17:32:01,2021-09-02 19:08:34,2021-09-02 19:09:21,2021-09-02 19:08:22,2021-09-02 19:11:24,2021-09-02 19:12:11,2021-09-02 19:11:04,2021-11-15 11:24:38,0.0,5760.0,21.0,162.0,5922.0,96.0,5826.0,1.69,Kein Signalschild,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5529,e423f489-0263-43e6-af52-aa577ca0e340,7aac3f05-0dea-401f-ae6d-c9936ac53d71,1878.0,2.0,4.20,4.20,4.20,False,STANDARD,1008,9008,completed,5034.0,2021-09-20 14:20:57,2021-09-20 14:20:57,2021-09-20 14:20:57,13135.0,2021-09-20 17:56:52,2021-09-20 17:59:52,2021-09-20 14:23:57,2021-09-20 18:00:32,2021-09-20 18:02:06,2021-09-20 18:01:08,2021-09-20 18:05:48,2021-09-20 18:07:22,2021-09-20 18:07:24,2021-11-15 11:24:38,0.0,12955.0,76.0,376.0,13331.0,225.0,13106.0,1.67,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10192,b1231cb6-2fbd-4809-a997-87b5114e342f,d2a75ace-8b9f-4b32-9914-85e434ce9019,3048.0,1.0,3.45,3.45,3.45,False,STANDARD,4007,1002,completed,7491.0,2021-12-01 12:14:37,2021-12-01 12:14:37,2021-12-01 12:14:37,3459.0,2021-12-01 13:00:32,2021-12-01 13:12:16,2021-12-01 12:17:37,2021-12-01 12:21:57,2021-12-01 13:15:41,2021-12-01 13:12:17,2021-12-01 12:28:40,2021-12-01 13:22:13,2021-12-01 13:21:34,2021-12-01 13:21:35,524.0,3279.0,1.0,557.0,3836.0,366.0,3470.0,1.52,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12494,a610e98b-8b66-4d12-bfcd-5c9ccfb30491,ce755d22-cfba-49c3-b37a-623f3c52cf7d,4185.0,5.0,12.65,12.20,12.20,False,BAHN_CARD,4026,1008,completed,4532.0,2021-08-30 11:55:43,2021-08-30 11:55:43,2021-08-30 11:55:43,13166.0,2021-08-30 15:32:09,2021-08-30 15:35:09,2021-08-30 11:58:43,2021-08-30 15:34:28,2021-08-30 15:51:37,2021-08-30 15:45:15,2021-08-30 15:44:53,2021-08-30 16:02:02,2021-08-30 15:58:27,2021-11-15 11:24:38,0.0,12986.0,606.0,792.0,13778.0,502.0,13276.0,1.58,Gelbes Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13597,1b4c4917-c58f-49

In [17]:
df_clean = pd.merge(df_c,df_test, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
df_clean

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index
0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483.0,1.0,4.65,4.65,0.00,False,STANDARD,11009,6004,completed,NaN,2021-06-30 23:12:47,2021-07-01 07:30:00,2021-07-01 07:22:00,637.0,2021-07-01 07:29:37,2021-07-01 07:32:37,2021-07-01 07:25:00,2021-07-01 07:33:55,2021-07-01 07:34:07,2021-07-01 07:33:08,2021-07-01 07:44:27,2021-07-01 07:44:39,2021-07-01 07:44:44,2021-11-03 16:00:44,0.0,457.0,31.0,696.0,1153.0,658.0,495.0,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bb916271-0627-4196-8ec1-5324e0e1f71d,f07028da-ca7e-4713-9e45-743c71712e80,3040.0,1.0,3.45,1.55,1.55,False,VRN,10003,6004,completed,NaN,2021-07-01 09:21:39,2021-07-01 09:21:39,2021-07-01 09:21:39,779.0,2021-07-01 09:31:38,2021-07-01 09:34:38,2021-07-01 09:24:39,2021-07-01 09:32:07,2021-07-01 09:36:59,2021-07-01 09:35:40,2021-07-01 09:40:59,2021-07-01 09:45:53,2021-07-01 09:42:27,2021-11-15 11:24:38,0.0,599.0,62.0,407.0,1006.0,365.0,641.0,1.12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f797b4d6-2fd4-442a-aec3-32c7f34c9b3e,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,6511.0,1.0,5.20,3.10,3.10,False,VRN,1009,5002,completed,NaN,2021-07-01 17:11:11,2021-07-01 17:11:11,2021-07-01 17:11:11,645.0,2021-07-01 17:18:56,2021-07-01 17:21:56,2021-07-01 17:14:11,2021-07-01 17:24:39,2021-07-01 17:23:01,2021-07-01 17:22:06,2021-07-01 17:36:25,2021-07-01 17:34:47,2021-07-01 17:32:41,2021-11-15 11:24:38,0.0,465.0,10.0,635.0,1100.0,781.0,319.0,0.81,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5b9be6cf-1b25-43d7-b077-e0ede60ba49a,f07028da-ca7e-4713-9e45-743c71712e80,3040.0,1.0,3.45,1.55,1.55,False,VRN,6004,10003,completed,NaN,2021-07-01 18:24:13,2021-07-01 18:24:13,2021-07-01 18:24:13,489.0,2021-07-01 18:29:22,2021-07-01 18:32:22,2021-07-01 18:27:13,2021-07-01 18:30:25,2021-07-01 18:34:21,2021-07-01 18:33:23,2021-07-01 18:38:27,2021-07-01 18:42:23,2021-07-01 18:39:50,2021-11-15 11:24:38,0.0,309.0,61.0,387.0,696.0,365.0,331.0,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90ef7121-ff20-40bb-9489-f9e935519443,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,5974.0,1.0,4.90,2.80,2.80,False,VRN,5002,9010,completed,NaN,2021-07-01 18:35:04,2021-07-01 18:35:04,2021-07-01 18:35:04,1135.0,2021-07-01 18:51:18,2021-07-01 18:53:59,2021-07-01 18:38:04,2021-07-01 18:53:37,2021-07-01 18:55:14,2021-07-01 18:53:59,2021-07-01 19:04:18,2021-07-01 19:05:55,2021-07-01 19:04:05,2021-11-15 11:24:38,-19.0,955.0,0.0,606.0,1561.0,717.0,844.0,0.85,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8997,4ebacda2-8da6-47a4-b2bb-fefb4df667a0,a41793c0-fb54-44c6-b877-4412ef66d9ac,3523.0,2.0,6.89,4.74,4.74,False,VRN,1008,12008,completed,NaN,2022-04-30 17:49:24,2022-04-30 19:00:00,2022-04-30 18:52:00,234.0,2022-04-30 18:51:03,2022-04-30 18:55:54,2022-04-30 18:55:00,2022-04-30 18:53:10,2022-04-30 19:04:04,2022-04-30 19:03:29,2022-04-30 19:00:27,2022-04-30 19:11:09,2022-04-30 19:11:39,2022-04-30 19:11:56,111.0,54.0,455.0,490.0,544.0,423.0,121.0,1.16,NaN,NaN,NaN,NaN,NaN,NaN

In [18]:
df_clean.to_excel(r'/Users/ericchittka/Downloads/data_cleaned.xlsx')

In [ ]:
drives = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
drives = drives[(drives["state"] == "completed")]
edges = pd.read_excel(f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

In [ ]:
df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name='MoDstops')

In [ ]:
startdate = drives.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = drives.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [ ]:
drives_w_simulation = pd.DataFrame(columns=drives.columns)
new_rides_all = pd.DataFrame(columns=drives.columns)
for i in range(1, 13):
    new_rides = generateRideSpecs(drives, drives_w_simulation, df_stops, edges, 10000, i, 2021)
    new_rides_all = pd.concat([new_rides, new_rides_all])

new_rides_all["simulated"] = True
drives["simulated"] = False

new_rides_all = pd.concat([drives, new_rides_all])



In [ ]:
drives["new"] = drives["pickup_address"].astype(str) +"_"+ drives["dropoff_address"].astype(str)

In [ ]:
drives["new"]

In [ ]:
new_rides_all[new_rides_all["simulated"] == True]

In [ ]:
aggregated_drives = calculate_drives(drives, start_date, end_date).reset_index()
aggregated_drives_sim = calculate_drives(new_rides_all, start_date, end_date).reset_index()

In [ ]:
aggregated_drives

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4), dpi=80)
plt.bar(aggregated_drives["index"],aggregated_drives["number_of_drives"])

plt.show()

In [ ]:

plt.figure(figsize=(12, 4), dpi=80)
plt.bar(aggregated_drives_sim["index"],aggregated_drives_sim["number_of_drives"])
plt.show()

In [ ]:
aggregated_drives_sim[aggregated_drives_sim["pickup_address"] == aggregated_drives_sim["dropoff_address"]]

In [ ]:
len(new_rides_all[(new_rides_all["pickup_address"] == 1007) & (new_rides_all["dropoff_address"] == 4037)])

In [ ]:
len(drives[(drives["pickup_address"] == 1007) & (drives["dropoff_address"] == 4037)])

In [ ]:
find_id_for_name("Rathaus Hambach", df_stops)

In [ ]:
graph_sim = calculate_graph(aggregated_drives_sim)
graph = calculate_graph(aggregated_drives)

In [ ]:
hotspots = get_hotspots(edges, aggregated_drives)

In [ ]:
hotspots = [spot[0] for spot in hotspots]

In [ ]:
hotspots

In [ ]:
hotspots = [1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]
drone_spots = [15011, 13001, 2002, 11007, 4016, 1002, 3020, 9019, 9005]

In [ ]:
graph_with_drones = calculate_graph(drives_w_flights)

In [ ]:
result1, a = get_shortest_ride(1001, 10001, graph)
result2, b = get_shortest_ride(1001, 10001, graph_sim)

In [ ]:
result1

In [ ]:
result2

In [ ]:
b